# PROJECT 4. 네이버 영화리뷰 감성 분석 도전하기
## 1) 데이터 수집 및 분석
### (1) 데이터 수집
데이터 셋은 네이버 영화의 댓글을 모아 구성된 **[Naver sentiment movie corpus](https://github.com/e9t/nsmc)** 입니다.  

In [36]:
import pandas as pd
import urllib.request
%matplotlib inline
import matplotlib.pyplot as plt
import re
from konlpy.tag import Okt
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences
from collections import Counter

# 데이터 읽기
train_data = pd.read_table('~/aiffel/exploration/sentiment_classification/ratings_train.txt')
test_data = pd.read_table('~/aiffel/exploration/sentiment_classification/ratings_test.txt')

### (2) 데이터 분석

In [37]:
print("train 데이터 개수 :", len(train_data))
print("test 데이터 개수 :", len(test_data))
print("데이터 label 개수 :", len((set(train_data["label"]))))

train_data.head(10)   # 데이터셋 10개 출력

train 데이터 개수 : 150000
test 데이터 개수 : 50000
데이터 label 개수 : 2


,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1
5,5403919,막 걸음마 뗀 3세부터 초등학교 1학년생인 8살용영화.ㅋㅋㅋ...별반개도 아까움.,0
6,7797314,원작의 긴장감을 제대로 살려내지못했다.,0
7,9443947,별 반개도 아깝다 욕나온다 이응경 길용우 연기생활이몇년인지..정말 발로해도 그것보단...,0
8,7156791,액션이 없는데도 재미 있는 몇안되는 영화,1
9,5912145,왜케 평점이 낮은건데? 꽤 볼만한데.. 헐리우드식 화려함에만 너무 길들여져 있나?,1


데이터 셋은 영화에 대한 리뷰와 그에 대한 평가(0:부정, 1:긍정)입니다.  
훈련 데이터는 150,000개 이고, 평가 데이터는 50,000개입니다.  
그리고 데이터 라벨은 총 2개입니다.  

## 2) 데이터 전처리
수집한 데이터들 중 일부는 중복값을 가지고 있거나 혹은 의미 없는 값을 가지고 있을 수도 있습니다.  
이런 불필요한 데이터들을 처리하기 위해 데이터 전처리를 합니다.  
자연어에서의 데이터 전처리는 토큰화, 정제, 정규화, 불용어 제거 등이 있습니다.  
### (1) 데이터 정제 (cleaning)
> 데이터 정제 : 데이터의 빈값(결측치)이나 정삼 범위를 벗어난 값(이상치)들을 제거하거나 다른 값으로 대체합니다.
<br/>

train 데이터와 test 데이터에 중복이 있는지 확인하고, 중복 데이터를 제거합니다.   
그리고 null값을 가진 데이터가 있는지 확인하고, 결측치(Nan) 데이터를 제거합니다.  
마지막으로 정규 표현식을 사용하여 구두점이나 특수문자, 공백을 제거합니다.


In [38]:
# 중복 제거하기
print("====================중복 제거====================")
print("중복 제거 전 train 데이터 개수 :", len(train_data))
train_data.drop_duplicates(subset=['document'], inplace=True) # document 열에서 중복인 내용이 있다면 중복 제거
print("중복 제거 후 train 데이터 개수 :", len(train_data))

print()

print("중복 제거 전 test 데이터 개수 :", len(test_data))
test_data.drop_duplicates(subset=['document'], inplace=True) # document 열에서 중복인 내용이 있다면 중복 제거
print("중복 제거 후 test 데이터 개수 :", len(test_data))
print("===============================================\n")

# null 제거하기
print("==================결측치 제거====================")
print("결측치 제거 전 train 데이터 개수 :", len(train_data))
train_data = train_data.dropna(how = 'any') # Null 값이 존재하는 행 제거
print("결측치 제거 후 train 데이터 개수 :", len(train_data))

print()

print("결측치 제거 전 test 데이터 개수 :", len(test_data))
test_data = test_data.dropna(how = 'any') # Null 값이 존재하는 행 제거
print("결측치 제거 후 test 데이터 개수 :", len(test_data))
print("===============================================\n")

# 정규 표현식
train_data["document"] = train_data["document"].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")    # 한글과 공백을 제외하고 모두 제거
test_data["document"] = test_data["document"].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")    # 한글과 공백을 제외하고 모두 제거

train_data = train_data.reset_index(drop=True)    # 인덱스 재정렬
test_data = test_data.reset_index(drop=True)    # 인덱스 재정렬

train_data

====================중복 제거====================
중복 제거 전 train 데이터 개수 : 150000
중복 제거 후 train 데이터 개수 : 146183

중복 제거 전 test 데이터 개수 : 50000
중복 제거 후 test 데이터 개수 : 49158

==================결측치 제거====================
결측치 제거 전 train 데이터 개수 : 146183
결측치 제거 후 train 데이터 개수 : 146182

결측치 제거 전 test 데이터 개수 : 49158
결측치 제거 후 test 데이터 개수 : 49157



,id,document,label
0,9976970,아 더빙 진짜 짜증나네요 목소리,0
1,3819312,흠포스터보고 초딩영화줄오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 솔직히 재미는 없다평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화스파이더맨에서 늙어보이기만 했던 커스틴 던...,1
...,...,...,...
146177,6222902,인간이 문제지 소는 뭔죄인가,0
146178,8549745,평점이 너무 낮아서,1
146179,9311800,이게 뭐요 한국인은 거들먹거리고 필리핀 혼혈은 착하다,0
146180,2376369,청춘 영화의 최고봉방황과 우울했던 날들의 자화상,1


train 중복 데이터 개수 : 3817개  
test 중복 데이터 개수 : 842개  
train 결측 데이터 개수 : 1개  
test 결측 데이터 개수 : 1개  
  
**총 train 데이터 개수 : 146182개**  
**총 test 데이터 개수 : 49157개**  


### (2) 데이터 토큰화 (tokenization) 및 불용어(Stopwords) 제거
> 토큰화 : 텍스트를 문장이나 단어로 각각 분리합니다.  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
예를 들어, "I like an apple." 를 토큰화하면 "I", "like", "an", "apple", "." 이 됩니다.
<br/>

> 불용어 제거 : 불용어란 데이터 셋에 자주 등장하지만 분석에 큰 의미는 갖지 않는 단어를 말합니다.  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
&nbsp;불용어가 다수 포함되어 있을수록 효율 감소, 처리시간 증가 등 악영향이 발생하므로 불용어를 제거합니다.

Mecab 형태소 분석기를 사용하여 데이터 토큰화를 합니다.  
그리고 데이터 토큰화를 하면서 불용어를 제거하도록 하겠습니다.  


In [39]:
from konlpy.tag import Mecab

tokenizer = Mecab()    # Mecab 형태소 분석기 불러오기 
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']    # 불용어

# 데이터 토큰화하기
x_train = []    # 토큰화 된 훈련 데이터를 저장할 리스트
for review in train_data["document"]:
    temp_x_train = tokenizer.morphs(review)    # 형태소 분석
    temp_x_train = [word for word in temp_x_train if not word in stopwords]    # 형태소 분석을 통해 나온 단어들 중 불용어를 제외한 단어들 저장
    x_train.append(temp_x_train)
    
# 데이터 토큰화하기
x_test = []    # 토큰화 된 훈련 데이터를 저장할 리스트
for review in test_data["document"]:
    temp_x_test = tokenizer.morphs(review)    # 형태소 분석
    temp_x_test = [word for word in temp_x_test if not word in stopwords]    # 형태소 분석을 통해 나온 단어들 중 불용어를 제외한 단어들 저장
    x_test.append(temp_x_test)
    
print("토큰화한 리뷰 5개 :")
for i in range(5):
    print(x_train[i][:])

토큰화한 리뷰 5개 :
['아', '더', '빙', '진짜', '짜증', '나', '네요', '목소리']
['흠', '포스터', '보고', '초딩', '영화', '줄', '오버', '연기', '조차', '가볍', '지', '않', '구나']
['너무', '재', '밓었다그래서보는것을추천한다']
['교도소', '이야기', '구먼', '솔직히', '재미', '없', '다', '평점', '조정']
['사이몬페그', '익살', '스런', '연기', '돋보였', '던', '영화', '스파이더맨', '에서', '늙', '어', '보이', '기', '만', '했', '던', '커스틴', '던스트', '너무나', '이뻐', '보였', '다']


### (3) 정수 인코딩
딥러닝 모델의 입력은 0과 1의 비트로 표현 되어있는 숫자만 가능합니다.  
그래서 기계가 텍스트를 숫자로 처리할 수 있도록 데이터에 정수 인코딩을 수행해야 합니다.  
토큰화된 데이터를 정수로 인코딩하도록 하겠습니다.

In [40]:
# 리뷰에 있는 단어들 중 사전에 없는 단어는 <UNK>로 바꾸기
def wordlist_to_indexlist(wordlist):
    return [word_to_index[word] if word in word_to_index else word_to_index['<UNK>'] for word in wordlist]


# 단어 빈도수 세기
words = np.concatenate(x_train).tolist()
counter = Counter(words)
counter = counter.most_common(10000-4)
vocab = ['<PAD>', '<BOS>', '<UNK>', '<UNUSED>'] + [key for key, _ in counter]

word_to_index = {word:index for index, word in enumerate(vocab)}    # 사전(단어:인덱스)
index_to_word = {index:word for word, index in word_to_index.items()}    # 사전(인덱스:단어)

x_train = list(map(wordlist_to_indexlist, x_train))
x_test = list(map(wordlist_to_indexlist, x_test))

top5 = [index_to_word[i+4] for i in range(50)]    # 단어 빈도수 top 100

print("사전에 등록된 단어 개수 :", len(word_to_index), "개")

print("\n=====단어 빈도수 TOP50=====")
for i in range(len(top5)):
    print("{}등 : {}".format(i+1, top5[i]))
print("========================\n")


사전에 등록된 단어 개수 : 10000 개

=====단어 빈도수 TOP50=====
1등 : 영화
2등 : 다
3등 : 고
4등 : 하
5등 : 을
6등 : 보
7등 : 게
8등 : 지
9등 : 있
10등 : 없
11등 : 좋
12등 : 나
13등 : 었
14등 : 만
15등 : 는데
16등 : 너무
17등 : 봤
18등 : 적
19등 : 안
20등 : 정말
21등 : 로
22등 : 것
23등 : 음
24등 : 아
25등 : 네요
26등 : 어
27등 : 재밌
28등 : 지만
29등 : 같
30등 : 진짜
31등 : 에서
32등 : 했
33등 : 기
34등 : 네
35등 : 않
36등 : 점
37등 : 거
38등 : 았
39등 : 수
40등 : 되
41등 : 면
42등 : ㅋㅋ
43등 : 인
44등 : 말
45등 : 연기
46등 : 주
47등 : 최고
48등 : 내
49등 : 평점
50등 : 이런



### (4) 패딩 (Padding)
> 패딩 : 가변적 길이를 가지는 문장을 같은 길이로 맞춰주는 작업
<br/>

자연어 처리를 하다보면 데이터의 각 문장은 서로 길이가 다를 수 있습니다.  
기계는 데이터를 행렬로 보기 때문에, 문장들의 길이를 전부 동일하게 해주는 패딩 작업을 해야합니다.  


In [42]:
# 패딩 길이 정하기

len_result = [len(s) for s in x_train]

print("리뷰의 최대 길이 :", np.max(len_result))
print("리뷰의 평균 길이 :", np.mean(len_result))

# 최대 길이 = (평균 + 2*표준편차) 
max_tokens = np.mean(len_result) + 2 * np.std(len_result)
maxlen = int(max_tokens)
print('pad_sequences maxlen : ', maxlen)

리뷰의 최대 길이 : 83
리뷰의 평균 길이 : 13.725663898427987
pad_sequences maxlen :  36


In [43]:
# 패딩 추가하기
x_train = keras.preprocessing.sequence.pad_sequences(x_train,
                                                        value=word_to_index["<PAD>"],
                                                        padding='pre', # 혹은 'pre'
                                                        maxlen=maxlen)

x_test = keras.preprocessing.sequence.pad_sequences(x_test,
                                                       value=word_to_index["<PAD>"],
                                                       padding='pre', # 혹은 'pre'
                                                       maxlen=maxlen)
y_train = np.array(list(train_data['label']))
y_test = np.array(list(test_data['label']))
print(x_train.shape)
print(x_train[0])

(146182, 36)
[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0  27  67 895  33 214  15  28 699]


리뷰의 길이를 모두 36으로 맞춰준 것을 확인할 수 있습니다.  
또한 길이가 36이 안되는 리뷰는 앞쪽에 <PAD> 값을 넣어 길이를 36으로 맞춘 것을 확인할 수 있습니다.

## 4) 모델링 및 학습
전처리된 데이터를 이용하여 딥러닝 모델을 학습시킵니다.  
딥러닝 모델은 3가지(RNN, CNN, LST)를 사용할 것입니다.  

### (1) 모델링 (Modeling)
> CNN : Convolution Neural Network의 약자로 합성곱 연산을 사용하는 인공신경망 중 하나입니다.  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
합성곱이란 하나의 함수와 또 다른 함수를 반전 이동한 값을 곱한 다음, 구간에 대해 적분하여 새로운 함수를 구하는 연산자입니다.
<br/>

> RNN : Recurrent Neural Netowrk의 약자로 입력과 출력을 시퀀스 단위로 처리하는 모델입니다.
<br/>

> LSTM : Long Short-Term Memory의 약자로 RNN의 장기 의존성 문제를 보완한 모델입니다. 
<br/>


In [180]:
# 딥러닝 모델 설계하기
from tensorflow.keras.layers import SimpleRNN, Embedding, Dense
from tensorflow.keras.models import Sequential

vocab_size = 10000    # 어휘 사전의 크기입니다(10,000개의 단어)
word_vector_dim = 14  # 워드 벡터의 차원수

# LSTM 모델
lstm_model = keras.Sequential()
lstm_model.add(keras.layers.Embedding(vocab_size, word_vector_dim, input_shape=(None,)))
lstm_model.add(keras.layers.LSTM(128))
lstm_model.add(keras.layers.Dense(128, activation='relu'))
lstm_model.add(keras.layers.Dense(1, activation='sigmoid'))

lstm_model.summary()
print()

# CNN 모델
cnn_model = keras.Sequential()
cnn_model.add(keras.layers.Embedding(vocab_size, word_vector_dim, input_shape=(None,)))
cnn_model.add(keras.layers.Conv1D(128, 1, activation='relu'))
cnn_model.add(keras.layers.MaxPooling1D())
cnn_model.add(keras.layers.Conv1D(256, 1, activation='relu'))
cnn_model.add(keras.layers.GlobalMaxPooling1D())
cnn_model.add(keras.layers.Dense(128, activation='relu'))
cnn_model.add(keras.layers.Dense(1, activation='sigmoid'))

cnn_model.summary()
print()

# RNN 모델
rnn_model = keras.Sequential()
rnn_model.add(keras.layers.Embedding(vocab_size, word_vector_dim, input_shape=(None,)))
rnn_model.add(SimpleRNN(125))
rnn_model.add(keras.layers.Dense(128, activation='relu'))
rnn_model.add(Dense(1, activation='sigmoid'))

rnn_model.summary()
print()

Model: "sequential_109"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_109 (Embedding)    (None, None, 14)          140000    
_________________________________________________________________
lstm_38 (LSTM)               (None, 128)               73216     
_________________________________________________________________
dense_187 (Dense)            (None, 128)               16512     
_________________________________________________________________
dense_188 (Dense)            (None, 1)                 129       
Total params: 229,857
Trainable params: 229,857
Non-trainable params: 0
_________________________________________________________________

Model: "sequential_110"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_110 (Embedding)    (None, None, 14)          140000    
_____________________

### (2) 학습 (train)
위에서 설계한 모델들(RNN, CNN, LSTM)을 사용하여 학습시킵니다.


In [181]:
# model 훈련전, 훈련용 데이터 셋 146182건 중 30000건을 분리하여 검증셋으로 사용합니다.

# validation set 30000건 분리
x_val = x_train[:10000]   
y_val = y_train[:10000]

# validation set을 제외한 나머지 
partial_x_train = x_train[10000:]  
partial_y_train = y_train[10000:]

print(partial_x_train.shape)
print(partial_y_train.shape)

(136182, 36)
(136182,)


In [182]:
# 모델 학습
epochs=3

# RNN 모델 학습
rnn_model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
              
rnn_history = rnn_model.fit(partial_x_train,
                    partial_y_train,
                    epochs=epochs,
                    batch_size=512,
                    validation_data=(x_val, y_val),
                    verbose=1)

rnn_results = rnn_model.evaluate(x_test,  y_test, verbose=2)

# CNN 모델 학습
cnn_model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

cnn_history = cnn_model.fit(partial_x_train,
                    partial_y_train,
                    epochs=epochs,
                    batch_size=512,
                    validation_data=(x_val, y_val),
                    verbose=1)

cnn_results = cnn_model.evaluate(x_test,  y_test, verbose=2)

# LSTM 모델 학습
lstm_model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

lstm_history = lstm_model.fit(partial_x_train,
                    partial_y_train,
                    epochs=epochs,
                    batch_size=512,
                    validation_data=(x_val, y_val),
                    verbose=1)

lstm_results = lstm_model.evaluate(x_test,  y_test, verbose=2)

print()
print("RNN 정확도 :", rnn_results[1])
print("CNN 정확도 :", cnn_results[1])
print("LSTM 정확도 :", lstm_results[1])

Epoch 1/3
266/266 [==============================] - 4s 15ms/step - loss: 0.4663 - accuracy: 0.7639 - val_loss: 0.3574 - val_accuracy: 0.8414
Epoch 2/3
266/266 [==============================] - 4s 15ms/step - loss: 0.3374 - accuracy: 0.8539 - val_loss: 0.3541 - val_accuracy: 0.8428
Epoch 3/3
266/266 [==============================] - 4s 15ms/step - loss: 0.3078 - accuracy: 0.8700 - val_loss: 0.3678 - val_accuracy: 0.8407
1537/1537 - 4s - loss: 0.3842 - accuracy: 0.8336
Epoch 1/3
266/266 [==============================] - 12s 43ms/step - loss: 0.4400 - accuracy: 0.7903 - val_loss: 0.3623 - val_accuracy: 0.8394
Epoch 2/3
266/266 [==============================] - 2s 6ms/step - loss: 0.3409 - accuracy: 0.8517 - val_loss: 0.3621 - val_accuracy: 0.8380
Epoch 3/3
266/266 [==============================] - 1s 6ms/step - loss: 0.3125 - accuracy: 0.8664 - val_loss: 0.3558 - val_accuracy: 0.8455
1537/1537 - 4s - loss: 0.3666 - accuracy: 0.8379
Epoch 1/3
266/266 [==============================] 

**정확도는 LSTM > CNN > RNN 순으로 높습니다. 하지만 거의 비슷한 정확도입니다.**  
**3가지 모델 중 LSTM의 정확도가 가장 높지만, 우리가 원하고자 정확도(85%이상)에는 미치지 못합니다.** 

## 5) 학습된 임베딩 레이어 분석
gensim을 활용하여 자체학습된 임베딩 레이어를 분석합니다.

In [183]:
import os
from gensim.models.keyedvectors import Word2VecKeyedVectors

# RNN 임베딩 레이어 분석
embedding_layer = rnn_model.layers[0]
weights = embedding_layer.get_weights()[0]

rnn_word2vec_file_path = os.getenv('HOME')+'/aiffel/exploration/sentiment_classification/rnn_word2vec.txt'
f = open(rnn_word2vec_file_path, 'w')
f.write('{} {}\n'.format(vocab_size-4, word_vector_dim))

vectors = rnn_model.get_weights()[0]
for i in range(4,vocab_size):
    f.write('{} {}\n'.format(index_to_word[i], ' '.join(map(str, list(vectors[i, :])))))
f.close()

rnn_word_vectors = Word2VecKeyedVectors.load_word2vec_format(rnn_word2vec_file_path, binary=False)

# CNN 임베딩 레이어 분석
embedding_layer = cnn_model.layers[0]
weights = embedding_layer.get_weights()[0]

cnn_word2vec_file_path = os.getenv('HOME')+'/aiffel/exploration/sentiment_classification/cnn_word2vec.txt'
f = open(cnn_word2vec_file_path, 'w')
f.write('{} {}\n'.format(vocab_size-4, word_vector_dim))

vectors = cnn_model.get_weights()[0]
for i in range(4,vocab_size):
    f.write('{} {}\n'.format(index_to_word[i], ' '.join(map(str, list(vectors[i, :])))))
f.close()

cnn_word_vectors = Word2VecKeyedVectors.load_word2vec_format(cnn_word2vec_file_path, binary=False)

# LSTM 임베딩 레이어 분석
embedding_layer = lstm_model.layers[0]
weights = embedding_layer.get_weights()[0]

lstm_word2vec_file_path = os.getenv('HOME')+'/aiffel/exploration/sentiment_classification/lstm_word2vec.txt'
f = open(lstm_word2vec_file_path, 'w')
f.write('{} {}\n'.format(vocab_size-4, word_vector_dim))

vectors = lstm_model.get_weights()[0]
for i in range(4,vocab_size):
    f.write('{} {}\n'.format(index_to_word[i], ' '.join(map(str, list(vectors[i, :])))))
f.close()

lstm_word_vectors = Word2VecKeyedVectors.load_word2vec_format(lstm_word2vec_file_path, binary=False)


In [184]:
print("RNN 차원 :", len(rnn_word_vectors['영화']))
print("CNN 차원 :", len(cnn_word_vectors['영화']))
print("LSTM 차원 :", len(lstm_word_vectors['영화']))
print()

temp = rnn_word_vectors.similar_by_word("최고")
print("(RNN)'최고'라는 단어와 유사한 단어들 :", [temp[i][0] for i in range(len(temp))])
temp = cnn_word_vectors.similar_by_word("최고")
print("(CNN)'최고'라는 단어와 유사한 단어들 :", [temp[i][0] for i in range(len(temp))])
temp = lstm_word_vectors.similar_by_word("최고")
print("(LSTM)'최고'라는 단어와 유사한 단어들 :", [temp[i][0] for i in range(len(temp))])
print()

temp = rnn_word_vectors.similar_by_word("지루")
print("(RNN)'지루'라는 단어와 유사한 단어들 :", [temp[i][0] for i in range(len(temp))])
temp = cnn_word_vectors.similar_by_word("지루")
print("(CNN)'지루'라는 단어와 유사한 단어들 :", [temp[i][0] for i in range(len(temp))])
temp = lstm_word_vectors.similar_by_word("지루")
print("(LSTM)'지루'라는 단어와 유사한 단어들 :", [temp[i][0] for i in range(len(temp))])
print()

RNN 차원 : 14
CNN 차원 : 14
LSTM 차원 : 14

(RNN)'최고'라는 단어와 유사한 단어들 : ['담백', '수작', '지연', '최상', '굿', '각지', '클레이', '웰메이드', '일어서', '명품']
(CNN)'최고'라는 단어와 유사한 단어들 : ['주몽', '한꺼번에', '각지', '고든', '레고', '그곳', '손색', '슬펐', '후딱', '사나']
(LSTM)'최고'라는 단어와 유사한 단어들 : ['흥겨운', '비로소', '하나하나', '김현주', '담백', '맛깔', '기막힌', '부패', '고조', '십수']

(RNN)'지루'라는 단어와 유사한 단어들 : ['철없', '유치', '썰렁', '으으으', '임요환', '대견', '잣', '빼앗', '싫', '애매']
(CNN)'지루'라는 단어와 유사한 단어들 : ['려니', '려구', '글쎄요', '비호', '환불', '책임', '아까', '불면증', '수치심', '드러워']
(LSTM)'지루'라는 단어와 유사한 단어들 : ['망쳤', '돌렸', '경력', '별루', '문란', '오유', '끝냈', '루지', '지루함', '멀미']



**RNN - '최고' : '굿', '최상' 등 유사한 단어가 있습니다. 하지만 그 외의 단어는 유사한 단어가 아닌것 같습니다.**  
**CNN - '최고' : 모든 단어가 전혀 관련이 없는 단어들인 것같습니다.**  
**LSTM - '최고' : 모든 단어가 전혀 관련이 없는 단어들인 것같습니다.**  

**RNN - '지루' : '썰렁', '유치' 등 유사한 단어가 있습니다. 하지만 그 외의 단어는 유사한 단어가 아닌것 같습니다.**  
**CNN - '지루' : 전혀 관련이 없는 단어들인 것같습니다.**  
**LSTM - '지루' : '지루함', '멀미' 등 유사한 단어가 있습니다. 하지만 그 외의 단어는 유사한 단어가 아닌것 같습니다.**  

## 6) 한국어 Word2Vec 임베딩 활용하기
워드 벡터를 더 정교하게 학습시키기 위해서 **[한국어 Word2Vec](https://github.com/Kyubyong/wordvectors)** 사이트에 있는 사전학습된 워드 임베딩 모델을 가져와서 활용해 보도록 하겠습니다.   

### (1) 워드투벡터 (Word2Vec)

In [185]:
import gensim
from gensim.models import Word2Vec

word2vec_path = os.getenv('HOME')+'/aiffel/exploration/sentiment_classification/ko/ko.bin'
word2vec = Word2Vec.load(word2vec_path)

print("Word2Vec 차원 :", len(word2vec.wv["영화"]))
print()

temp = word2vec.wv.most_similar("최고")
print("'최고'라는 단어와 유사한 단어들 :", [temp[i][0] for i in range(len(temp))])

temp = word2vec.wv.most_similar("지루")
print("'지루'라는 단어와 유사한 단어들 :", [temp[i][0] for i in range(len(temp))])

Word2Vec 차원 : 200

'최고'라는 단어와 유사한 단어들 : ['최대', '최강', '유일한', '일류', '최악', '최연소', '랭킹', '제일의', '최초', '최상']
'지루'라는 단어와 유사한 단어들 : ['답답', '편안', '솔직', '쓸쓸', '차분', '조용', '냉정', '자유분방', '피곤', '느긋']


In [186]:
vocab_size = 10000    # 어휘 사전의 크기 (10,000개의 단어)
word_vector_dim = 200  # 워드 벡터의 차원수

embedding_matrix = np.random.rand(vocab_size, word_vector_dim)

# embedding_matrix에 Word2Vec 워드벡터를 단어 하나씩마다 차례차례 카피한다.
for i in range(4,vocab_size):
    if index_to_word[i] in word2vec.wv:
        embedding_matrix[i] = word2vec.wv[index_to_word[i]]

### (2) 모델링 (Modeling)

In [187]:
# 딥러닝 모델 설계하기
from tensorflow.keras.initializers import Constant

vocab_size = 10000    # 어휘 사전의 크기입니다(10,000개의 단어)
word_vector_dim = 200  # 워드 벡터의 차원수

# LSTM 모델
lstm_model = keras.Sequential()
lstm_model.add(keras.layers.Embedding(vocab_size, word_vector_dim, input_shape=(None,)))
lstm_model.add(keras.layers.LSTM(128))
lstm_model.add(keras.layers.Dense(128, activation='relu'))
lstm_model.add(keras.layers.Dense(1, activation='sigmoid'))

lstm_model.summary()
print()

# CNN 모델
cnn_model = keras.Sequential()
cnn_model.add(keras.layers.Embedding(vocab_size, word_vector_dim, input_shape=(None,)))
cnn_model.add(keras.layers.Conv1D(128, 1, activation='relu'))
cnn_model.add(keras.layers.MaxPooling1D())
cnn_model.add(keras.layers.Conv1D(256, 1, activation='relu'))
cnn_model.add(keras.layers.GlobalMaxPooling1D())
cnn_model.add(keras.layers.Dense(128, activation='relu'))
cnn_model.add(keras.layers.Dense(1, activation='sigmoid'))

cnn_model.summary()
print()

# RNN 모델
rnn_model = keras.Sequential()
rnn_model.add(keras.layers.Embedding(vocab_size, word_vector_dim, input_shape=(None,)))
rnn_model.add(SimpleRNN(125))
rnn_model.add(keras.layers.Dense(128, activation='relu'))
rnn_model.add(Dense(1, activation='sigmoid'))

rnn_model.summary()
print()

Model: "sequential_112"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_112 (Embedding)    (None, None, 200)         2000000   
_________________________________________________________________
lstm_39 (LSTM)               (None, 128)               168448    
_________________________________________________________________
dense_193 (Dense)            (None, 128)               16512     
_________________________________________________________________
dense_194 (Dense)            (None, 1)                 129       
Total params: 2,185,089
Trainable params: 2,185,089
Non-trainable params: 0
_________________________________________________________________

Model: "sequential_113"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_113 (Embedding)    (None, None, 200)         2000000   
_________________

### (2) 학습 (train)

In [188]:
# 모델 학습
epochs=3

# RNN 모델 학습
rnn_model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

rnn_history = rnn_model.fit(partial_x_train,
                    partial_y_train,
                    epochs=epochs,
                    batch_size=512,
                    validation_data=(x_val, y_val),
                    verbose=1)

rnn_results = rnn_model.evaluate(x_test,  y_test, verbose=2)

# CNN 모델 학습
cnn_model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

cnn_history = cnn_model.fit(partial_x_train,
                    partial_y_train,
                    epochs=epochs,
                    batch_size=512,
                    validation_data=(x_val, y_val),
                    verbose=1)

cnn_results = cnn_model.evaluate(x_test,  y_test, verbose=2)

# LSTM 모델 학습
lstm_model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

lstm_history = lstm_model.fit(partial_x_train,
                    partial_y_train,
                    epochs=epochs,
                    batch_size=512,
                    validation_data=(x_val, y_val),
                    verbose=1)

lstm_results = lstm_model.evaluate(x_test,  y_test, verbose=2)

print()
print("RNN 정확도 :", rnn_results[1])
print("CNN 정확도 :", cnn_results[1])
print("LSTM 정확도 :", lstm_results[1])


Epoch 1/3
266/266 [==============================] - 10s 36ms/step - loss: 0.4299 - accuracy: 0.7934 - val_loss: 0.3486 - val_accuracy: 0.8435
Epoch 2/3
266/266 [==============================] - 9s 35ms/step - loss: 0.3103 - accuracy: 0.8663 - val_loss: 0.3735 - val_accuracy: 0.8357
Epoch 3/3
266/266 [==============================] - 9s 35ms/step - loss: 0.2233 - accuracy: 0.9089 - val_loss: 0.4223 - val_accuracy: 0.8330
1537/1537 - 5s - loss: 0.4337 - accuracy: 0.8266
Epoch 1/3
266/266 [==============================] - 12s 44ms/step - loss: 0.4137 - accuracy: 0.8076 - val_loss: 0.3518 - val_accuracy: 0.8456
Epoch 2/3
266/266 [==============================] - 6s 23ms/step - loss: 0.3208 - accuracy: 0.8627 - val_loss: 0.3447 - val_accuracy: 0.8467
Epoch 3/3
266/266 [==============================] - 6s 23ms/step - loss: 0.2789 - accuracy: 0.8844 - val_loss: 0.3584 - val_accuracy: 0.8473
1537/1537 - 3s - loss: 0.3686 - accuracy: 0.8415
Epoch 1/3
266/266 [=============================

## [ 결과  - 루브릭] 
| Model | Word2Vec 사용전 | Word2Vec 사용후 | accuracy 변화율 |
|:----------|:----------:|:----------:|:----------:|
| RNN | 83.36% | 82.66% | -0.7% |
| CNN | 83.79% | 84.15% | +0.36% |
| LSTM | 83.91% | 84.96% | +1.05% |  
<br/>

#### 1. 다양한 방법으로 Text Classification 태스크를 성공적으로 구현하였다.
- 3가지 이상의 모델(RNN, LSTM,, CNN)을 성공적으로 시도하였습니다. :)

#### 2. gensim을 활용하여 자체학습된 혹은 사전학습된 임베딩 레이어를 분석하였다.
- gensim의 유사단어 찾기를 활용하여 사전학습된 임베딩을 분석하였습니다. :)

#### 3. 한국어 Word2Vec을 활용하여 가시적인 성능향상을 달성했다.
- 네이버 영화리뷰 데이터 감성분석 정확도를 85% 이상 달성하지 못했습니다. :(  
    *(ps. LSTM의 정확도가 84.96%인데... 반올림하면 85%인데... 85%라고 해도 될 것 같은데... ㅠ.ㅠ)*

## [느낀점]
Word2Vec을 사용했음에도 불구하고, 정확도가 많이 향상되지 않았습니다.  
그래서 정확도가 왜 많이 향상되지 않았을까 그 이유에 대해 생각을 해보았습니다.  
#### 1. 불용어를 더 제거해야한다고 생각합니다.  
    위의 소스코드에서 빈도수 단어 TOP5를 출력했을때, '다', '수', '인', '을' 등 긍부정을 분류하기에는 좋지 않은 단어들이 있었습니다.
    이러한 단어들이 긍부정을 분류할때, 방해가 된것 같습니다.  
    제 생각에는 조사나 접미사 같은 단어들을 더 제거해야 높은 정확도를 얻을 수 있다고 생각합니다.
#### 2. 긍부정을 분류할 때, 문장안에 있는 단어의 유사도보다는 문장이 어떤 유형의 글인지가 더 중요하다고 생각합니다.
    영화와 관련된 문장, 스포츠와 관련된 문장, 연예인과 관련된 문장 등 세상에는 여러 카테고리의 문장들이 있습니다.  
    각 카테고리별로 문장의 긍부정을 분류할 때, 긍부정을 판별하는 feature는 서로 다를 수 있습니다.  
    예를 들어, '대박', '쩐다'와 같은 단어는 영화리뷰에서 긍정단어로 사용되지만, 연예인관련기사에서는 부정단어로 사용될 수 있습니다.  
    왜냐하면 "진짜 대박이네..." 같은 문장은 연예인에게 실망했을때 쓰일 수 있고, "오바쩐다." 같은 문장은 연예인을 안좋게 볼때 쓰일 수 있기 때문입니다. 
    
> **정확도 85%를 달성하지 못해서 아쉽지만, 그로 인해 깨닫게 된 것들이 많이 있어서 도움이 되었습니다. :)**

정확도를 올리려고 validation set, epoch, model input 등 다 바꿔봤는데도 안오르네요....  
정말 힘든 프로젝트였습니다... ㅠㅠ